<a href="https://colab.research.google.com/github/leotodisco/SmolLM-Fine-Tuning/blob/main/SmolLM_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qqq datasets==3.2.0 transformers==4.47.1 trl==0.14.0 peft==0.14.0 accelerate==1.2.1 bitsandbytes==0.45.2 --progress-bar off
!pip install -qqq flash-attn --no-build-isolation --progress-bar off


In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import GRPOConfig, GRPOTrainer
from transformers import BitsAndBytesConfig

# Load The Dataset

## Task: reddit posts summarization

In [ ]:
dataset = load_dataset("mlabonne/smoltldr")

In [ ]:
dataset["train"]["prompt"][0]

"SUBREDDIT: r/tifu\n\nTITLE: TIFU by trying to pet a dog.\n\nPOST: Last night I went to a Hippie May Day Festival/ Camp out. Needless to say, I passed out hard in my tent at the end of the night.Woken by the warmth and light of the morning sun, I emerged from my tent in search of some water to quench my burgeoning thirst. To my delight I spotted a dog scouting the field before me, about 110 meters away. Without delay I dashed towards it, my urge to pet this dog was immeasurable. On the way back to my tent, while running, I just so happened to come upon the most heinous stick I have ever encountered. The bastard was sticking straight out of the earth, cleverly hidden in a plush, verdant meadow. My foot never saw it coming. It had no warning, no shoe, no defense! The poor soul (no pun intended) never saw it coming, until the moment it was impaled by what I would have thought was the devils pitchfork itself. The worst part is, I didn't even get to pet the dog, it sprinted back to it's own

In [ ]:
dataset["train"]["completion"][0]

' Tried to pet a dog, foot got impaled by a demon stick, never even got to pet the dog.'

The dataset is composed of:
- Prompts
- Completions

We have 2000 prompts and 2000 completions


In [ ]:
model_name = "HuggingFaceTB/SmolLM-135M-Instruct"

bnb_config = BitsAndBytesConfig(
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model = prepare_model_for_kbit_training(model)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
"""!sudo add-apt-repository ppa:ubuntu-toolchain-r/test
!sudo apt-get update
!sudo apt-get install gcc-4.9
!sudo apt-get upgrade libstdc++6"""

'!sudo add-apt-repository ppa:ubuntu-toolchain-r/test\n!sudo apt-get update\n!sudo apt-get install gcc-4.9\n!sudo apt-get upgrade libstdc++6'

In [ ]:
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules="all-linear"
)
model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

trainable params: 2,442,240 || all params: 136,957,248 || trainable%: 1.7832
None


## Reward Function


In [ ]:
# Reward function
ideal_length = 50


def reward_len(completions, **kwargs):
    return [-abs(ideal_length - len(completion)) for completion in completions]

In [ ]:
training_args = GRPOConfig(
    output_dir="GRPO",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    max_prompt_length=512,
    max_completion_length=96,
    num_generations=8,
    optim="adamw_8bit",
    num_train_epochs=1,
    bf16=True,
    remove_unused_columns=False,
    logging_steps=1,
    report_to=["none"]
)

In [ ]:
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[reward_len],
    args=training_args,
    train_dataset=dataset["train"],
)


trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
